In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


# The first three lines are invalid -> skip
journeys = pd.read_csv('../data/train-drives-cleaned.csv', encoding='utf-8')

# Select relevant columns
journeys_filtered = journeys.loc[:, journeys.columns.str.startswith('train_line') | journeys.columns.isin(['planned_departure_hour', 'day_of_week'])]
# journeys_filtered = journeys.loc[:, journeys.columns.isin(['planned_departure_hour', 'day_of_week'])]
class_label_df = journeys['crowdedness']

# Split into train and test sets
journeys_train, journeys_test, class_label_train, class_label_test = train_test_split(journeys_filtered, class_label_df, test_size=0.2, random_state=123)
journeys_filtered.head()


In [ ]:
############################################ MANUAL CHECK REQUIRED !!! ############################################

# Plot the journeys by planned departure time (grouped per hour) on x and crowdedness on y
# Normalize the counts per hour

# Group by hour and crowdedness, count occurrences
journeys_visual = journeys.groupby(['planned_departure_hour', 'crowdedness']).size().reset_index(name='count')

# Normalize the counts per hour
journeys_visual['normalized_count'] = journeys_visual.groupby('planned_departure_hour')['count'].transform(lambda x: x / x.sum())

import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(7, 4))
sns.barplot(data=journeys_visual, x='planned_departure_hour', y='normalized_count', hue='crowdedness', palette='viridis')
plt.title('Normalized Train Journeys by Planned Departure Hour and Crowdedness')
plt.xlabel('Planned Departure Hour')
plt.ylabel('Normalized Number of Journeys')
plt.legend(title='Crowdedness')
plt.xticks(rotation=45)
plt.tight_layout()




# make a plot which shows the total number of journeys grouped by hour
journeys_total = journeys.groupby('planned_departure_hour').size().reset_index(name='total_count')
plt.figure(figsize=(7, 4))
sns.barplot(data=journeys_total, x='planned_departure_hour', y='total_count', palette ='Blues')
plt.title('Total Number of Train Journeys by Planned Departure Hour')
plt.xlabel('Planned Departure Hour')
plt.ylabel('Total Number of Journeys')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

class_labels = class_label_train.unique().astype(str)
gini_index = DecisionTreeClassifier(criterion='gini', random_state=123)
gini_index.fit(journeys_train, class_label_train)

plt.figure(figsize=(40, 20))
plot_tree(gini_index, filled=True, feature_names=journeys_train.columns, class_names=class_labels, rounded=True)
plt.title('Decision Tree Classifier (Gini Index)')
plt.show()

# Predict and evaluate the model
predictions_gini = gini_index.predict(journeys_test)
accuracy_gini = accuracy_score(class_label_test, predictions_gini)
print(f'Accuracy of Gini Index Decision Tree: {accuracy_gini:.2f}')
print(classification_report(class_label_test, predictions_gini, target_names=class_labels))


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Set up the model
clf = DecisionTreeClassifier(random_state=123)

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=clf,
                           param_grid=param_grid,
                           cv=5,               # 5-fold cross-validation
                           scoring='accuracy',
                           n_jobs=-1)          # Use all CPU cores

# Fit
grid_search.fit(journeys_train, class_label_train)

print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)


plt.figure(figsize=(40, 20))
plot_tree(grid_search.best_estimator_, filled=True, feature_names=journeys_train.columns, class_names=class_labels, rounded=True)
plt.title('Decision Tree Classifier (Gini Index)')
plt.show()


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# Scale the features (optional, but often beneficial for KNN)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
journeys_train_scaled = scaler.fit_transform(journeys_train)
journeys_test_scaled = scaler.transform(journeys_test)


# As seen in visuals, it is imbalance, use smote
# Apply SMOTE only to the training set
smote = SMOTE(random_state=123, k_neighbors=3)
journeys_train_balanced, class_label_train_balanced = smote.fit_resample(journeys_train_scaled, class_label_train)

# Try different values of k and use cross-validation to find the best one
k_range = range(1, 21)
cv_scores = []

for k in k_range:
    knn_classifier = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn_classifier, journeys_train_balanced, class_label_train_balanced, cv=3, scoring='accuracy')
    cv_scores.append(scores.mean())

best_k = k_range[cv_scores.index(max(cv_scores))]
print(f'Best k found by cross-validation: {best_k}')

# Re-initialize KNN classifier with the best k
knn_classifier = KNeighborsClassifier(n_neighbors=3)

# Fit the model
knn_classifier.fit(journeys_train_balanced, class_label_train_balanced)

# Predict on the test set
knn_predictions = knn_classifier.predict(journeys_test_scaled)

# Evaluate the KNN model
knn_accuracy = accuracy_score(class_label_test, knn_predictions)
print(f'Accuracy of KNN Classifier: {knn_accuracy:.3f}')
print(classification_report(class_label_test, knn_predictions, target_names=class_labels))
